<a href="https://colab.research.google.com/github/mbenedicto99/RUNDECK_AI/blob/main/Rundeck_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Lê SOMENTE arquivos .csv do diretório 'data' e gera ./app/app/ai_analysis.json.

Entrada (em data/):
  - execucoes.csv  (campos: projeto, job, exec_id, inicio, status, duracao_s, …)
  - score.csv      (campos: exec_id, re)

Saída:
  - app/app/ai_analysis.json

Uso:
  python scripts/build_ai_json.py \
    --data-dir data \
    --out app/app/ai_analysis.json
"""
from __future__ import annotations
import argparse
import json
from pathlib import Path
import sys
import pandas as pd
import numpy as np


def _fail(msg: str, code: int = 2):
    print(f"ERRO: {msg}", file=sys.stderr)
    sys.exit(code)


def load_csvs(data_dir: Path) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Carrega execucoes.csv e score.csv exclusivamente de data_dir."""
    if not data_dir.exists():
        _fail(f"Diretório não encontrado: {data_dir}")

    # Somente .csv permitidos
    extra = [p for p in data_dir.iterdir() if p.is_file() and p.suffix.lower() != ".csv"]
    if extra:
        _fail(f"Somente .csv são aceitos em {data_dir}. Remova/ou mova: "
              + ", ".join(sorted(p.name for p in extra)))

    exec_path = data_dir / "execucoes.csv"
    score_path = data_dir / "score.csv"

    if not exec_path.exists():
        _fail(f"Arquivo obrigatório não encontrado: {exec_path}")
    if not score_path.exists():
        _fail(f"Arquivo obrigatório não encontrado: {score_path}")

    # Leitura robusta
    df_exec = pd.read_csv(exec_path, dtype=str, keep_default_na=False, na_values=["", "NA", "NaN"])
    df_score = pd.read_csv(score_path, dtype=str, keep_default_na=False, na_values=["", "NA", "NaN"])

    # Tipagens mínimas / normalização
    # exec
    for col in ["projeto", "job", "exec_id", "inicio", "status", "duracao_s"]:
        if col not in df_exec.columns:
            _fail(f"Coluna obrigatória ausente em execucoes.csv: {col}")

    # cast duracao_s -> float
    df_exec["duracao_s"] = pd.to_numeric(df_exec["duracao_s"], errors="coerce")
    # parse datetime se possível
    if "inicio" in df_exec.columns:
        try:
            df_exec["inicio"] = pd.to_datetime(df_exec["inicio"], errors="coerce")
        except Exception:
            pass

    # score
    if "exec_id" not in df_score.columns or "re" not in df_score.columns:
        _fail("Colunas obrigatórias ausentes em score.csv: exec_id, re")
    df_score["re"] = pd.to_numeric(df_score["re"], errors="coerce")

    # limpar ids
    df_exec["exec_id"] = df_exec["exec_id"].astype(str).str.strip()
    df_score["exec_id"] = df_score["exec_id"].astype(str).str.strip()

    # descartar linhas inválidas
    df_exec = df_exec[df_exec["exec_id"].notna() & (df_exec["exec_id"] != "")]
    df_score = df_score[df_score["exec_id"].notna() & (df_score["exec_id"] != "")]
    df_score = df_score[df_score["re"].notna()]

    return df_exec, df_score


def build_analysis(df_exec: pd.DataFrame, df_score: pd.DataFrame) -> dict:
    """Gera dicionário com resumo, hotspots, p95 por job e amostras."""
    # join por exec_id
    df = df_exec.merge(df_score[["exec_id", "re"]], on="exec_id", how="left")

    # métricas básicas
    total = len(df)
    por_status = df["status"].fillna("desconhecido").value_counts(dropna=False).to_dict()
    duracao_med = float(np.nanmean(df["duracao_s"])) if "duracao_s" in df else None
    re_p95_global = float(np.nanpercentile(df["re"], 95)) if df["re"].notna().any() else None

    resumo = {
        "total_execucoes": int(total),
        "por_status": por_status,
        "duracao_media_s": None if np.isnan(duracao_med) else duracao_med,
        "re_p95_global": re_p95_global,
    }

    # risco p95 por job (usando 'projeto' + 'job' quando existirem)
    chave_job = ["projeto", "job"] if all(c in df.columns for c in ["projeto", "job"]) else ["job"]
    gb = df.dropna(subset=["re"]).groupby(chave_job)["re"]
    risco_p95_por_job = (
        gb.quantile(0.95)
        .reset_index()
        .rename(columns={"re": "re_p95"})
        .sort_values("re_p95", ascending=False)
        .head(200)
        .to_dict(orient="records")
    )

    # hotspots: top execuções por re
    hotspots = (
        df.dropna(subset=["re"])
          .sort_values("re", ascending=False)
          .loc[:, ["projeto", "job", "exec_id", "inicio", "status", "duracao_s", "re"]]
          .head(50)
    )
    # serialização amigável
    def _ser(dt):
        if pd.isna(dt):
            return None
        if hasattr(dt, "isoformat"):
            return dt.isoformat()
        return dt

    hotspots_serial = [
        {
            "projeto": r.get("projeto"),
            "job": r.get("job"),
            "exec_id": r.get("exec_id"),
            "inicio": _ser(r.get("inicio")),
            "status": r.get("status"),
            "duracao_s": None if pd.isna(r.get("duracao_s")) else float(r.get("duracao_s")),
            "re": float(r.get("re")),
        }
        for r in hotspots.to_dict(orient="records")
    ]

    # amostras (curto): top 100
    top_amostras = hotspots_serial[:100]

    return {
        "resumo": resumo,
        "risco_p95_por_job": risco_p95_por_job,
        "hotspots": hotspots_serial,
        "top_amostras": top_amostras,
    }


def main():
    ap = argparse.ArgumentParser(description="Gera ai_analysis.json a partir de .csv em 'data/'.")
    ap.add_argument("--data-dir", default="data", help="Diretório de entrada (somente .csv).")
    ap.add_argument("--out", default="app/app/ai_analysis.json", help="Arquivo de saída JSON.")
    args = ap.parse_args()

    data_dir = Path(args.data_dir)
    out_path = Path(args.out)

    df_exec, df_score = load_csvs(data_dir)
    result = build_analysis(df_exec, df_score)

    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    print(json.dumps({
        "status": "ok",
        "out": str(out_path),
        "resumo": result["resumo"],
        "counts": {
            "hotspots": len(result["hotspots"]),
            "risco_p95_por_job": len(result["risco_p95_por_job"]),
            "top_amostras": len(result["top_amostras"])
        }
    }, ensure_ascii=False))


if __name__ == "__main__":
    main()